In [1]:
import os
initial_dir = os.getcwd()
print(initial_dir)

/home/kalmar/Mathematics/Research/MPI Tuebingen/TDA-Cause-Effect-Pairs


In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import re
import sys
import time

sys.path.append("/home/kalmar/Mathematics/Research/MPI Tuebingen/Dionysus-python3/build/bindings/python")

%reload_ext autoreload
%autoreload 2

import dionysus

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

import TDA
tsp = __import__("topology-summary-plotter")
out_i = __import__("identify-outliers")
sampler = __import__("points-sampler")
os.chdir(initial_dir)

In [4]:
pattern = re.compile('pair00..\.txt')
files = sorted([x for x in os.listdir('./pairs/') if pattern.match(x)])
file_names = []
dir_names = []
for f in files:
    with open("./pairs/" + f) as points:
        x = points.readline()
        if len(x.split()) > 2:
            file_names.append(f)
            dir_names.append(f[:-4])
            
for i, f in enumerate(file_names):
    x = np.loadtxt('./pairs/'+f)
    print(i, f, x.shape)

0 pair0052.txt (10226, 8)
1 pair0053.txt (989, 4)
2 pair0054.txt (392, 5)
3 pair0055.txt (72, 32)
4 pair0071.txt (120, 8)
5 pair0081.txt (365, 3)
6 pair0082.txt (365, 3)
7 pair0083.txt (365, 3)


In [5]:
i = 0
points = np.loadtxt('./test/'+dir_names[i]+"/std_points")
print(points.shape)
points[35]

(3000, 8)


array([-1.12157885, -1.12804363, -1.13075389, -1.13338204, -1.13692543,
       -1.12943448, -1.12384011, -1.1242145 ])

In [6]:
xrange= range(4,8)
yrange= range(0,4)
g_complex = TDA.GeometricComplex(points[:500], xrange, yrange, full_initialisation=False)

INFO:root:Creating GeometricComplex on 500 points
INFO:root:Variable X range: 4 5 6 7
INFO:root:Variable Y range: 0 1 2 3
INFO:root:Using Rips-complex with radius 2.828427. This may be slow for dense sets!
INFO:root:Created rips full complex of size 58709
INFO:root:The threshold 2.104506 limits the complex size to 44615


In [7]:
%lprun -f TDA.GeometricComplex.filtered_complex f_complex = g_complex.filtered_complex(3)

In [8]:
%prun f_complex = g_complex.filtered_complex(3)

In [9]:
import pickle

In [11]:
pickle_string = pickle.dumps(g_complex.limited_complex)

In [12]:
len(pickle_string)

40

In [13]:
print(pickle_string)

b'\x80\x03cdionysus._dionysus\nFiltration\nq\x00)Rq\x01.'
